<a href="https://colab.research.google.com/github/liuxiao916/Dive_into_DL_PyTorch/blob/main/3_9_%E5%A4%9A%E5%B1%82%E6%84%9F%E7%9F%A5%E6%9C%BA%E7%9A%84%E4%BB%8E%E9%9B%B6%E5%BC%80%E5%A7%8B%E7%9A%84%E5%AE%9E%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3.9 多层感知机的从零开始的实现

In [1]:
%matplotlib inline
import torch
import torchvision
import numpy as np

### 3.9.1 读取数据集

In [2]:
Batch_size =256
fashionmnist_train = torchvision.datasets.FashionMNIST(root = './data/fashionMNIST', train = True, transform=torchvision.transforms.ToTensor(),download=True)
fashionmnist_train_dataloader = torch.utils.data.DataLoader(dataset= fashionmnist_train,batch_size = Batch_size, shuffle = True)
fashionmnist_test = torchvision.datasets.FashionMNIST(root = './data/fashionMNIST', train = False, transform=torchvision.transforms.ToTensor(),download=True)
fashionmnist_test_dataloader = torch.utils.data.DataLoader(dataset= fashionmnist_test,batch_size = Batch_size, shuffle = True)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### 3.9.2 定义模型参数

In [3]:
num_inputs, num_outputs, num_hiddens = 784, 10, 256

W1 = torch.tensor(np.random.normal(scale = 0.01, size = (num_inputs, num_hiddens)), dtype=torch.float, requires_grad=True)
b1 = torch.zeros(num_hiddens, dtype=torch.float, requires_grad=True)
W2 = torch.tensor(np.random.normal(scale = 0.01, size = (num_hiddens, num_outputs)), dtype=torch.float, requires_grad=True)
b2 = torch.zeros(num_outputs, dtype=torch.float, requires_grad=True)

params = [W1, b1, W2, b2]

### 3.9.3 定义激活函数

In [4]:
def relu(X):
    return torch.maximum(X, torch.tensor(0.0))

###  3.9.4 定义模型

In [5]:
def net(X):
    X = X.view((-1,num_inputs))
    H = relu(torch.mm(X,W1)+b1)
    return torch.mm(H,W2)+b2

### 3.9.5 定义损失函数

In [6]:
loss = torch.nn.CrossEntropyLoss()

### 3.9.6 训练模型

In [7]:
def sgd(params, lr, batch_size):
    for param in params:
        param.data = param.data - lr*param.grad / batch_size

In [8]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0 
    for X,y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum/n

In [9]:
num_epoch = 5
lr = 50

def train_ch3(net, train_iter,test_iter, loss, num_epochs, batch_size, params = None, lr = None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n =0.0, 0.0, 0
        for X,y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()
            l.backward()
            sgd(params, lr, batch_size)
            
            for param in params:
                param.grad.data.zero_()

            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f' % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

train_ch3(net, fashionmnist_train_dataloader,fashionmnist_test_dataloader, loss, num_epoch, Batch_size, params, lr )

        

epoch 1, loss 0.0034, train acc 0.691, test acc 0.727
epoch 2, loss 0.0021, train acc 0.814, test acc 0.825
epoch 3, loss 0.0018, train acc 0.836, test acc 0.826
epoch 4, loss 0.0017, train acc 0.847, test acc 0.829
epoch 5, loss 0.0016, train acc 0.857, test acc 0.849
